In [7]:
import pymc3 as pm
import matplotlib.pyplot as plt
import numpy as np
from utils import ECDF
from data import load_decay
import pandas as pd
import theano.tensor as tt

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('dark_background')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

Now that you've learned about Bayesian estimation, we're going to explore one more topic: Bayesian curve fitting.

By "curve fitting", we're really talking about any curve: those that are bendy, those that are straight, and those that are in between. 

In order to reinforce this point, rather than show you plain vanilla linear regression, we will work through an exponential decay curve example.

# Problem Setup

You've taken radioactive decay measurements of an unknown element in a secure facility. The measurements are noisy, though, and potentially have some bias. In the face of this, we would like to be able to characterize the decay constant of this unknown material, potentially leading to an identification of the material.

Let's load in the data.

In [8]:
np.random.seed(42)

df = load_decay()
df.head(5)

,t,activity
0,0,63.496714
1,1,62.281634
2,2,62.495498
3,3,62.806653
4,4,60.493075


**Exercise:** Plot `activity` vs. `time`.

In [4]:
ax = df['activity'].plot()

**Discuss:** 

- For the scenario that we're in, what is a plausible equation that links time to activity?
- What are the key parameters that we need to worry about?
- What might be justifiable priors for them?

## Link Functions

If we were to draw out a model for the curve above, it might look like this:

![](../images/radioactive-decay-model.jpg)

The most important part of this diagram is the "link function" - this is what "links" the data to the output. In this case, we've used the exponential decay curve as the link function, but if you were doing a linear regression model, all you would have to do is to change the link function for the $y=mx+c$ "straight curve", and do another curve fit with the appropriate priors for $m$ and $c$.

If you're familiar with the mathematical groundings of deep learning, you'll immediately recognize that a deep neural network model is merely another instance of a really complicated link function that links the input data $x$ to the observed data $y$, with the model weights and biases corresponding to the parameters (let's collectively call this set of parameters $\theta$).

**Exercise:** Now that you've seen a pictorial description of the model, implement it below in PyMC3.

In [9]:
with pm.Model() as model:
    A = pm.HalfNormal('A', sd=100)
    tau = pm.Exponential('tau', lam=1)
    C = pm.Normal('C', sd=100)
    
    
    sd = pm.HalfCauchy('sd', beta=1)
    
    link = A * np.exp(-df['t'].values / tau) + C
    
    like = pm.Normal('activity', mu=link, sd=sd, observed=df['activity'].values)

Sample from the posterior.

In [ ]:
with model:
    trace = pm.sample(2000, tune=2000)
    # Note: Sampler may pause for a while after finishing

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [sd_log__, C, tau_log__, A_log__]
 72%|████████████████████████████████████▏             | 2891/4000 [00:07<00:02, 382.19it/s]

Check that sampling has converged.

In [ ]:
traces = pm.traceplot(trace)

# Summary

- In lieu of showing you a "straight curve" (line) fit, you've now seen an arbitrary curve fit.
- As long as you can find a way to parameterize the curve with a function, you can perform inference on the curve's parameters.
- That function is called the "link function", which provides the link between the parameters, data and the output.

More generally, if

$$y = f(x, \theta)$$

where $\theta$ are merely a set of parameters, then you can perform inference on the curve's parameters $\theta$. To make this clear:

| curve name | functional form | parameters |
|------------|-----------------|---------------------|
| exponential decay | $y = Ae^{-t/\tau} + C$ | $A$, $\tau$, $C$|
| sine curves | $y = A\sin(\omega  x + \phi)$ | $A$, $\omega$, $\phi$ |
| linear regression | $y = mx + c$ | $m$, $c$ |
| logistic regression | $y = L(mx + c)$ | $m$, $c$ |
| 4-parameter IC50 | $y = \frac{a - i}{1 + 10^{\beta(log(\tau) - x)}} + i$ | $a$, $i$, $\tau$, $\beta$ |
| deep learning | $y = f(x, \theta)$ | $\theta$ |